In [ ]:
import base64
import io

from lxml import html
from tika import parser


def extract_text_from_pdf(file_path):
    parsed = parser.from_file(file_path, xmlContent=True)
    parsed_xml = parsed["content"]

    et = html.fromstring(parsed_xml)
    pages = et.getchildren()[1].getchildren()

    return [str(page.text_content()) for page in pages]

In [ ]:
INPUT_FOLDER = "raw"
OUTPUT_FOLDER = "json"

In [ ]:
try:
    os.stat(OUTPUT_FOLDER)
except:
    os.mkdir(OUTPUT_FOLDER)

In [ ]:
import json, os, re

In [ ]:
already_done = set([re.sub(".json$", "", f) for f in os.listdir(OUTPUT_FOLDER)])

In [ ]:
for root, folder, files in os.walk(INPUT_FOLDER):
    for file_name in files:
        if file_name in already_done:
            continue
        if not file_name.endswith("pdf"):
            continue
        
        full_file = root + "/" + file_name
        print(full_file)

        try:
            texts = extract_text_from_pdf(full_file)
        except:
            #print ("Error processing", full_file, ". Skipping")
            continue
            
        if len(texts) < 10:
            continue
        if len(" ".join(texts)) < 1000:
            continue
        
        output_file = OUTPUT_FOLDER + "/" + file_name + ".json"

        with open(output_file, 'w', encoding="utf-8") as fo:
            fo.write(json.dumps(texts))